# 0. Imports

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import matplotlib.gridspec as gridspec

from ydata_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from kneed import KneeLocator
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from collections import Counter
from tabulate import tabulate
from tsfeatures import tsfeatures
from contextlib import contextmanager
import sys
from mango import Tuner
import time

In [25]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.holtwinters import Holt
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [26]:
# Configure seaborn plot style: set background color and use dark grid
sns.set(rc={'axes.facecolor':'#E6E6E6'}, style='darkgrid')

In [27]:
df_train = pd.read_csv("data/train_clustered.csv", index_col=0)
df_test = pd.read_csv("data/test_clustered.csv", index_col=0)
cluster = pd.read_csv("data/clustered_products.csv", index_col=0)
clusters_model = pd.read_csv("data/clusters_model.csv", index_col=0)

In [28]:
df_train.ds = pd.to_datetime(df_train.ds, format="%Y-%m-%d")
df_test.ds = pd.to_datetime(df_test.ds, format="%Y-%m-%d")

In [29]:
nb_clusters = df_train.cluster.nunique()

In [30]:
clusters_model

,cluster,model,params
0,0,RandomForestRegressor,"{'n_estimators': 100, 'max_depth': 6}"
1,2,ARIMA,"{'q': 1, 'p': 1, 'd': 0}"
2,1,SARIMA,"{'D': 1, 'P': 1, 'Q': 0, 'd': 0, 'p': 2, 'q': ..."


# V. Forecasting

In [31]:
@contextmanager
def suppress_output():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        try:
            sys.stdout = devnull
            sys.stderr = devnull
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr

In [32]:
import importlib
import parameters

importlib.reload(parameters)
from parameters import param_spaces, optimizer_configs

def evaluate_model(train, test, model_name):
    """
    Optimize parameters for a given model.
    """

    def objective_function(args_list):
        results = []
        for params in args_list:
            try:
                if model_name == "SARIMA":
                    order = (params['p'], params['d'], params['q'])
                    seasonal_order = (params['P'], params['D'], params['Q'], params['s'])
                    model_instance = SARIMAX(train, order=order, seasonal_order=seasonal_order).fit(disp=False)
                    forecast = model_instance.forecast(steps=len(test))
                elif model_name == "ARIMA":
                    order = (params['p'], params['d'], params['q'])
                    model_instance = ARIMA(train, order=order).fit(disp=False)
                    forecast = model_instance.forecast(steps=len(test))
                elif model_name == "Exponential Smoothing":
                    model_instance = ExponentialSmoothing(
                        train,
                        seasonal=params['seasonal'],
                        seasonal_periods=params['seasonal_periods']
                    ).fit()
                    forecast = model_instance.forecast(steps=len(test))
                elif model_name == "Holt":
                    model_instance = Holt(train, exponential=params['exponential']).fit()
                    forecast = model_instance.forecast(steps=len(test))
                elif model_name == "XGBRegressor":
                    X_train = np.arange(len(train)).reshape(-1, 1)
                    model_instance = XGBRegressor(
                        n_estimators=params['n_estimators'],
                        max_depth=params['max_depth'],
                        learning_rate=params['learning_rate'],
                        objective='reg:squarederror',
                        random_state=42
                    )
                    model_instance.fit(X_train, train)
                    X_test = np.arange(len(train), len(train) + len(test)).reshape(-1, 1)
                    forecast = model_instance.predict(X_test)
                elif model_name == "RandomForestRegressor":
                    X_train = np.arange(len(train)).reshape(-1, 1)
                    model_instance = RandomForestRegressor(
                        n_estimators=params['n_estimators'],
                        max_depth=params['max_depth'],
                        random_state=42
                    )
                    model_instance.fit(X_train, train)
                    X_test = np.arange(len(train), len(train) + len(test)).reshape(-1, 1)
                    forecast = model_instance.predict(X_test)
                elif model_name == "LSTM":
                    model_instance = Sequential([
                        LSTM(params['units'], activation='relu', input_shape=(1, 1)),
                        Dense(1)
                    ])
                    model_instance.compile(optimizer='adam', loss='mae')
                    train_reshaped = train.values.reshape(-1, 1, 1)
                    model_instance.fit(train_reshaped, train, epochs=params['epochs'], batch_size=1, verbose=0)
                    test_reshaped = np.arange(len(train), len(train) + len(test)).reshape(-1, 1, 1)
                    forecast = model_instance.predict(test_reshaped).flatten()
                else:
                    forecast = train  # Placeholder in case no valid model is provided

                error = mean_absolute_error(test, forecast)
                results.append(error)
            except Exception as e:
                results.append(100000000)  # Assign a high error value for exceptions
        return results

    param_space = {}
    optimizer_config = {'initial_random': 5, 'num_iteration': 10}

    # Define parameter spaces for each model
    if model_name == "SARIMA":
        param_space = param_spaces[model_name]
        optimizer_config = optimizer_configs[model_name]
    elif model_name == "ARIMA":
        param_space = param_spaces[model_name]
        optimizer_config = optimizer_configs[model_name]
    elif model_name == "Exponential Smoothing":
        param_space = param_spaces[model_name]
    elif model_name == "Holt":
        param_space = param_spaces[model_name]
    elif model_name == "XGBRegressor":
        param_space = param_spaces[model_name]
        optimizer_config = optimizer_configs[model_name]
    elif model_name == "RandomForestRegressor":
        param_space = param_spaces[model_name]
        optimizer_config = optimizer_configs[model_name]
    elif model_name == "LSTM":
        param_space = param_spaces[model_name]
    else:
        raise ValueError(f"Unsupported model: {model_name}")

    # Run tuner
    tuner = Tuner(param_space, objective_function, optimizer_config)
    results = tuner.minimize()

    return results['best_params']

**Optimized Cluster Based Analysis for Model Selection**

In [33]:
# Check if timing file exists
if os.path.exists("timing.csv"):
    timing = pd.read_csv("timing.csv", index_col=0)
else:
    timing = pd.DataFrame(columns=['time_started', 'type of run', 'time_ended', 'time_elapsed'])

In [34]:
forecast_results = {}
timer = time.time()
timing_row_starting = [time.strftime("%H:%M:%S", time.gmtime(time.time())), 'forecasting w clustering', None, None]


for c in range(nb_clusters):
    items = df_train[df_train.cluster == c]["unique_id"].unique()
    best_model = clusters_model.loc[clusters_model.cluster == c, 'model'].squeeze()
    cluster_counter = 0
    for item in items:
        cluster_counter += 1
        train_serie = df_train[df_train.unique_id == item].y
        test_serie = df_test[df_test.unique_id == item].y
        if ((cluster_counter % 25) == 0):
            print(f"Proccessed {cluster_counter} items out of {len(items)} ({cluster_counter/len(items) * 100}%) in cluster {c} with model {best_model}.")
        # Optimize parameters for the best model
        with suppress_output():
            best_params = evaluate_model(train_serie, test_serie, best_model)

        # Use the best model with optimized parameters for forecasting
        if best_model == 'SARIMA':
            model = SARIMAX(
                train_serie,
                order=(best_params['p'], best_params['d'], best_params['q']),
                seasonal_order=(best_params['P'], best_params['D'], best_params['Q'], best_params['s'])
            ).fit()
            forecast = model.forecast(steps=len(test_serie))
        elif best_model == 'ARIMA':
            model = ARIMA(
                train_serie,
                order=(best_params['p'], best_params['d'], best_params['q'])
            ).fit()
            forecast = model.forecast(steps=len(test_serie))
        elif best_model == 'Exponential Smoothing':
            model = ExponentialSmoothing(
                train_serie,
                seasonal=best_params['seasonal'],
                seasonal_periods=best_params['seasonal_periods']
            ).fit()
            forecast = model.forecast(steps=len(test_serie))
        elif best_model == "Holt's Linear Trend":
            model = Holt(train_serie, exponential=best_params['exponential']).fit()
            forecast = model.forecast(steps=len(test_serie))
        elif best_model == 'XGBRegressor':
            X_train = np.arange(len(train_serie)).reshape(-1, 1)
            xgb_model = XGBRegressor(
                n_estimators=best_params['n_estimators'],
                max_depth=best_params['max_depth'],
                learning_rate=best_params['learning_rate'],
                objective='reg:squarederror',
                random_state=42
            )
            xgb_model.fit(X_train, train_serie)
            X_test = np.arange(len(train_serie), len(train_serie) + len(test_serie)).reshape(-1, 1)
            forecast = xgb_model.predict(X_test)
        elif best_model == 'RandomForestRegressor':
            X_train = np.arange(len(train_serie)).reshape(-1, 1)
            rf_model = RandomForestRegressor(
                n_estimators=best_params['n_estimators'],
                max_depth=best_params['max_depth'],
                random_state=42
            )
            rf_model.fit(X_train, train_serie)
            X_test = np.arange(len(train_serie), len(train_serie) + len(test_serie)).reshape(-1, 1)
            forecast = rf_model.predict(X_test)
        elif best_model == 'LSTM':
            lstm_model = Sequential([
                LSTM(best_params['units'], activation='relu', input_shape=(1, 1)),
                Dense(1)
            ])
            lstm_model.compile(optimizer='adam', loss='mae')
            train_reshaped = train_serie.values.reshape(-1, 1, 1)
            lstm_model.fit(train_reshaped, train_serie, epochs=best_params['epochs'], batch_size=1, verbose=0)
            test_reshaped = np.arange(len(train_serie), len(train_serie) + len(test_serie)).reshape(-1, 1, 1)
            forecast = lstm_model.predict(test_reshaped).flatten()

        forecast_results[item] = forecast

timing_row_ending = [None, 'forecasting w clustering', time.strftime("%H:%M:%S", time.gmtime(time.time())), time.time() - timer]

timing = pd.concat([timing, pd.DataFrame([timing_row_starting, timing_row_ending], columns=timing.columns)])


Proccessed 25 items out of 367 (6.811989100817439%) in cluster 0 with model RandomForestRegressor.
Proccessed 50 items out of 367 (13.623978201634879%) in cluster 0 with model RandomForestRegressor.
Proccessed 75 items out of 367 (20.435967302452315%) in cluster 0 with model RandomForestRegressor.
Proccessed 100 items out of 367 (27.247956403269757%) in cluster 0 with model RandomForestRegressor.
Proccessed 125 items out of 367 (34.05994550408719%) in cluster 0 with model RandomForestRegressor.
Proccessed 150 items out of 367 (40.87193460490463%) in cluster 0 with model RandomForestRegressor.
Proccessed 175 items out of 367 (47.68392370572207%) in cluster 0 with model RandomForestRegressor.
Proccessed 200 items out of 367 (54.495912806539515%) in cluster 0 with model RandomForestRegressor.
Proccessed 225 items out of 367 (61.30790190735694%) in cluster 0 with model RandomForestRegressor.
Proccessed 250 items out of 367 (68.11989100817438%) in cluster 0 with model RandomForestRegressor.

In [35]:
forecast_results
# Round to the closest integer
# To save to df 
# Comparaison of MAE

{'F00001015_CLR000021': array([53.18315975, 53.18315975, 53.18315975, 53.18315975, 53.18315975,
        53.18315975, 53.18315975, 53.18315975]),
 'F00001015_CLR000023': array([24.009499, 24.009499, 24.009499, 24.009499, 24.009499, 24.009499,
        24.009499, 24.009499]),
 'F00001111_151304TCX': array([0., 0., 0., 0., 0., 0., 0., 0.]),
 'F00001111_193924TPX': array([28.75358874, 28.75358874, 28.75358874, 28.75358874, 28.75358874,
        28.75358874, 28.75358874, 28.75358874]),
 'F00001111_CLR000021': array([44.90417318, 44.90417318, 44.90417318, 44.90417318, 44.90417318,
        44.90417318, 44.90417318, 44.90417318]),
 'F00001111_CLR000391': array([28.07266143, 28.07266143, 28.07266143, 28.07266143, 28.07266143,
        28.07266143, 28.07266143, 28.07266143]),
 'F00001112_CLR000023': array([0., 0., 0., 0., 0., 0., 0., 0.]),
 'F00001231_181017TCX': array([0.67480295, 0.67480295, 0.67480295, 0.67480295, 0.67480295,
        0.67480295, 0.67480295, 0.67480295]),
 'F00001231_CLR000021': 

In [36]:
df_test

,unique_id,ds,y,cluster
0,F00001015_CLR000021,2024-10-06,53.0,0.0
1,F00001015_CLR000023,2024-10-06,20.0,0.0
2,F00001111_151304TCX,2024-10-06,218.0,0.0
3,F00001111_193924TPX,2024-10-06,24.0,0.0
4,F00001111_CLR000021,2024-10-06,52.0,0.0
...,...,...,...,...
5595,M24400088_190414TCX,2024-11-24,0.0,NaN
5596,M24400088_CLR000021,2024-11-24,0.0,NaN
5597,M24900011_180515TCX,2024-11-24,0.0,2.0
5598,S00029_CLR001336,2024-11-24,0.0,0.0


In [37]:
df_forecast =pd.DataFrame(forecast_results)

In [38]:
df_forecast["ds"] = df_test.ds.unique()

In [39]:
df_forecast = df_forecast.melt(id_vars="ds",
                 var_name="unique_id",
                 value_name="forecast")

In [40]:
df_forecast["forecast"] = df_forecast.forecast.round(0).astype(int)

In [41]:
df_forecast = df_forecast[["unique_id", "ds", "forecast"]]

In [42]:
df_forecast.to_csv("data/forecast_cluster_optimizer.csv")

### Brute Force Method ###

In [44]:
from mango import Tuner
import parameters
import importlib

importlib.reload(parameters)
from parameters import param_spaces, optimizer_configs

def evaluate_models_brute_force(train, test):
    results = {}

    def objective_function(args_list):
        results = []
        for params in args_list:
            try:
                if model == "SARIMA":
                    order = (params['p'], params['d'], params['q'])
                    seasonal_order = (params['P'], params['D'], params['Q'], params['s'])
                    model_instance = SARIMAX(train, order=order, seasonal_order=seasonal_order).fit(disp=False)
                elif model == "ExponentialSmoothing":
                    model_instance = ExponentialSmoothing(
                        train,
                        seasonal=params['seasonal'],
                        seasonal_periods=params['seasonal_periods']
                    ).fit()
                elif model == "Holt":
                    model_instance = Holt(train, exponential=params['exponential']).fit()
                elif model == "LinearRegression":
                    X_train = np.arange(len(train)).reshape(-1, 1)
                    model_instance = LinearRegression()
                    model_instance.fit(X_train, train)
                    forecast = model_instance.predict(np.arange(len(train), len(train) + len(test)).reshape(-1, 1))
                    error = mean_absolute_error(test, forecast)
                    results.append(error)
                    continue
                elif model == "XGBRegressor":
                    X_train = np.arange(len(train)).reshape(-1, 1)
                    model_instance = XGBRegressor(
                        n_estimators=params['n_estimators'],
                        max_depth=params['max_depth'],
                        learning_rate=params['learning_rate'],
                        objective='reg:squarederror',
                        random_state=42
                    )
                    model_instance.fit(X_train, train)
                    forecast = model_instance.predict(np.arange(len(train), len(train) + len(test)).reshape(-1, 1))
                    error = mean_absolute_error(test, forecast)
                    results.append(error)
                    continue
                elif model == "ARIMA":
                    order = (params['p'], params['d'], params['q'])
                    model_instance = ARIMA(train, order=order).fit()
                elif model == "RandomForestRegressor":
                    X_train = np.arange(len(train)).reshape(-1, 1)
                    model_instance = RandomForestRegressor(
                        n_estimators=params['n_estimators'],
                        max_depth=params['max_depth'],
                        random_state=42
                    )
                    model_instance.fit(X_train, train)
                    forecast = model_instance.predict(np.arange(len(train), len(train) + len(test)).reshape(-1, 1))
                    error = mean_absolute_error(test, forecast)
                    results.append(error)
                    continue
                elif model == "LSTM":
                    model_instance = Sequential([
                        LSTM(params['units'], activation='relu', input_shape=(1, 1)),
                        Dense(1)
                    ])
                    model_instance.compile(optimizer='adam', loss='mae')
                    train_reshaped = train.values.reshape(-1, 1, 1)
                    model_instance.fit(train_reshaped, train, epochs=params['epochs'], batch_size=1, verbose=0)
                    test_reshaped = np.arange(len(train), len(train) + len(test)).reshape(-1, 1, 1)
                    forecast = model_instance.predict(test_reshaped).flatten()
                    error = mean_absolute_error(test, forecast)
                    results.append(error)
                    continue

                forecast = model_instance.forecast(steps=len(test))
                error = mean_absolute_error(test, forecast)
                results.append(error)
            except Exception as e:
                results.append(100000000)  # Assign a high error value for exceptions
        return results

    # SARIMA
    try:
        model = "SARIMA"
        param_space = param_spaces[model]
        optimizer_config = optimizer_configs[model]
        tuner = Tuner(param_space, objective_function, optimizer_config)
        sarima_results = tuner.minimize()

        best_params = sarima_results['best_params']
        best_order = (best_params['p'], best_params['d'], best_params['q'])
        best_seasonal_order = (best_params['P'], best_params['D'], best_params['Q'], best_params['s'])
        sarima_model = SARIMAX(train, order=best_order, seasonal_order=best_seasonal_order).fit(disp=False)
        sarima_forecast = sarima_model.forecast(steps=len(test))
        results['SARIMA'] = (mean_absolute_error(test, sarima_forecast), sarima_forecast)
    except Exception as e:
        results['SARIMA'] = float('inf')

    # Exponential Smoothing
    try:
        model = "ExponentialSmoothing"
        param_space = param_spaces[model]
        optimizer_config = optimizer_configs[model]
        tuner = Tuner(param_space, objective_function, optimizer_config)
        es_results = tuner.minimize()

        best_params = es_results['best_params']
        es_model = ExponentialSmoothing(
            train,
            seasonal=best_params['seasonal'],
            seasonal_periods=best_params['seasonal_periods']
        ).fit()
        es_forecast = es_model.forecast(steps=len(test))
        results['Exponential Smoothing'] = (mean_absolute_error(test, es_forecast), es_forecast)
    except Exception as e:
        results['Exponential Smoothing'] = float('inf')

    # Holt's Linear Trend
    try:
        model = "Holt"
        param_space = param_spaces[model]
        optimizer_config = optimizer_configs[model]
        tuner = Tuner(param_space, objective_function, optimizer_config)
        holt_results = tuner.minimize()

        best_params = holt_results['best_params']
        holt_model = Holt(train, exponential=best_params['exponential']).fit()
        holt_forecast = holt_model.forecast(steps=len(test))
        results["Holt's Linear Trend"] = (mean_absolute_error(test, holt_forecast), holt_forecast)
    except Exception as e:
        results["Holt's Linear Trend"] = float('inf')

    # XGBoost
    try:
        model = "XGBRegressor"
        param_space = param_spaces[model]
        optimizer_config = optimizer_configs[model]
        tuner = Tuner(param_space, objective_function, optimizer_config)
        xgb_results = tuner.minimize()

        best_params = xgb_results['best_params']
        X_train = np.arange(len(train)).reshape(-1, 1)
        X_test = np.arange(len(train), len(train) + len(test)).reshape(-1, 1)
        xgb_model = XGBRegressor(
            n_estimators=best_params['n_estimators'],
            max_depth=best_params['max_depth'],
            learning_rate=best_params['learning_rate'],
            objective='reg:squarederror',
            random_state=42
        )
        xgb_model.fit(X_train, train)
        xgb_forecast = xgb_model.predict(X_test)
        results['XGBoost'] = (mean_absolute_error(test, xgb_forecast), xgb_forecast)
    except Exception as e:
        results['XGBoost'] = float('inf')

    # ARIMA
    try:
        model = "ARIMA"
        param_space = param_spaces[model]
        optimizer_config = optimizer_configs[model]
        tuner = Tuner(param_space, objective_function, optimizer_config)
        arima_results = tuner.minimize()

        best_params = arima_results['best_params']
        best_order = (best_params['p'], best_params['d'], best_params['q'])
        arima_model = ARIMA(train, order=best_order).fit()
        arima_forecast = arima_model.forecast(steps=len(test))
        results['ARIMA'] = (mean_absolute_error(test, arima_forecast), arima_forecast)
    except Exception as e:
        results['ARIMA'] = float('inf')

    # Linear Regression
    try:
        model = "LinearRegression"
        param_space = {}  # No hyperparameters to tune for Linear Regression
        optimizer_config = {'initial_random': 1, 'num_iteration': 1}
        tuner = Tuner(param_space, objective_function, optimizer_config)
        lr_results = tuner.minimize()

        X_train = np.arange(len(train)).reshape(-1, 1)
        lr_model = LinearRegression()
        lr_model.fit(X_train, train)
        X_test = np.arange(len(train), len(train) + len(test)).reshape(-1, 1)
        lr_forecast = lr_model.predict(X_test)
        results['Linear Regression'] = (mean_absolute_error(test, lr_forecast), lr_forecast)
    except Exception as e:
        results['Linear Regression'] = float('inf')

    # Random Forest
    try:
        model = "RandomForestRegressor"
        param_space = param_spaces[model]
        optimizer_config = optimizer_configs[model]
        tuner = Tuner(param_space, objective_function, optimizer_config)
        rf_results = tuner.minimize()

        best_params = rf_results['best_params']
        X_train = np.arange(len(train)).reshape(-1, 1)
        X_test = np.arange(len(train), len(train) + len(test)).reshape(-1, 1)
        rf_model = RandomForestRegressor(
            n_estimators=best_params['n_estimators'],
            max_depth=best_params['max_depth'],
            random_state=42
        )
        rf_model.fit(X_train, train)
        rf_forecast = rf_model.predict(X_test)
        results['RandomForestRegressor'] = (mean_absolute_error(test, rf_forecast), rf_forecast)
    except Exception as e:
        results['RandomForestRegressor'] = float('inf')

    # LSTM
    try:
        model = "LSTM"
        param_space = param_spaces[model]
        optimizer_config = optimizer_configs[model]
        tuner = Tuner(param_space, objective_function, optimizer_config)
        lstm_results = tuner.minimize()

        best_params = lstm_results['best_params']
        lstm_model = Sequential([
            LSTM(best_params['units'], activation='relu', input_shape=(1, 1)),
            Dense(1)
        ])
        lstm_model.compile(optimizer='adam', loss='mae')
        train_reshaped = train.values.reshape(-1, 1, 1)
        lstm_model.fit(train_reshaped, train, epochs=best_params['epochs'], batch_size=1, verbose=0)
        test_reshaped = np.arange(len(train), len(train) + len(test)).reshape(-1, 1, 1)
        lstm_forecast = lstm_model.predict(test_reshaped).flatten()
        results['LSTM'] = (mean_absolute_error(test, lstm_forecast), lstm_forecast)
    except Exception as e:
        results['LSTM'] = float('inf')

    return results


In [ ]:
import parameters
importlib.reload(parameters)
from parameters import param_spaces, optimizer_configs

brute_force_forecast_stats = {}
brute_force_forecast_results = {}
timer = time.time()
timing_row_starting = [time.strftime("%H:%M:%S", time.gmtime(time.time())), 'forecasting bruteforce', None, None]
len_items = len(df_train.unique_id.unique())
counter = 0
for uniqe_item in df_train.unique_id.unique():
    counter += 1
    if ((cluster_counter % 25) == 0):
            print(f"Proccessed {counter} items out of {len_items}")
    train = df_train[df_train["unique_id"] == uniqe_item].y
    test = df_test[df_test["unique_id"] == uniqe_item].y
    with suppress_output():
        mae = evaluate_models_brute_force(train, test)
    filtered_mae = {k: v for k, v in mae.items() if isinstance(v, tuple)}
    best_model = min(filtered_mae.items(), key=lambda x: x[1][0])
    clusters_model[c] = best_model
    brute_force_forecast_stats[uniqe_item] = [best_model[0], best_model[1][0], best_model[1][1]]
    brute_force_forecast_results[uniqe_item] = best_model[0]
    if counter == 1:
         print(brute_force_forecast_results)
         print(brute_force_forecast_stats)
    #print(f"Item {uniqe_item}: Best Model = {best_model} (MAE = {mae[best_model][0]:.2f})")

timing_row_ending = [None, 'forecasting bruteforce', time.strftime("%H:%M:%S", time.gmtime(time.time())), time.time() - timer]
timing = pd.concat([timing, pd.DataFrame([timing_row_starting, timing_row_ending], columns=timing.columns)])

TypeError: 'float' object is not subscriptable

In [ ]:
timing.to_csv("timing.csv")

In [ ]:
brute_force_forecast_stats_df = pd.DataFrame.from_dict(brute_force_forecast_stats, orient='index', columns=['model', 'mae', 'forecast'])
brute_force_forecast_stats_df.index.name = 'unique_id'
brute_force_forecast_stats_df.to_csv("data/brute_force_forecast_stats.csv")

In [ ]:
df_forecast_bf =pd.DataFrame(brute_force_forecast_results)

In [ ]:
df_forecast_bf["ds"] = df_test.ds.unique()

In [ ]:
df_forecast_bf = df_forecast_bf.melt(id_vars="ds",
                 var_name="unique_id",
                 value_name="forecast")

In [ ]:
df_forecast_bf["forecast"] = df_forecast_bf.forecast.round(0).astype(int)

In [ ]:
df_forecast_bf = df_forecast_bf[["unique_id", "ds", "forecast"]]

In [ ]:
df_forecast_bf.to_csv("data/forecast_brute_force.csv")